[View in Colaboratory](https://colab.research.google.com/github/gowtham91m/Analytics_Vidhya_hackathon/blob/master/enigma_codefest_machine_learning.ipynb)

# **Problem Statement** 

Date : Aug 30 2018 

An online question and answer platform has hired you as a data scientist to identify the best question authors on the platform. This
identification will bring more insight into increasing the user engagement. Given the tag of the question, number of views received,
number of answers, username and reputation of the question author, the problem requires you to predict the upvote count that the
question will receive.

Linear model fitted with 3 degree polynomial features did good job on leader board and got me 14 th rank although variance is seen high in the cross validatoin score.


In [0]:
%%capture
!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.linear_model import LinearRegression, RidgeCV, Ridge,LassoCV, Lasso
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn.preprocessing import PolynomialFeatures
from google.colab import files
#from sklearn.cross_validation import *
from sklearn.model_selection import cross_val_score
from time import time

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
train = drive.CreateFile({'id': '10ihNrtkuPvwlVEODXmw9N2etRWBxTdmh'})
test = drive.CreateFile({'id': '10XhWYDZZYiW0jx8tzj0iijKiEFwUXTrk'})

train.GetContentFile('train.csv')
test.GetContentFile('test.csv')

In [0]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train_id = train.ID
test_id = test.ID
data = train.append(test)

dummies = pd.get_dummies(data.Tag,drop_first=True)
data = pd.concat([data,dummies],axis=1)
#data.Answers=np.log(data.Answers+1)
#data.Reputation=np.log(data.Reputation+1)
# drop username
data.drop(['ID','Username','Tag'],axis=1,inplace=True)

train = data.loc[~data.Upvotes.isnull()]
test=data.loc[data.Upvotes.isnull()]
test = test.drop(['Upvotes'],axis=1)

y = train.Upvotes
X = train.drop(['Upvotes'],axis=1)

#X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)
#dtrain = xgb.DMatrix(X_train, label=y_train)
#dtest = xgb.DMatrix(X_test, label=y_test)
#dtrain = xgb.DMatrix(X,y)
#test_matrix = xgb.DMatrix(test)
#Std = StandardScaler().fit(X)
#X = Std.transform(X)
#X_test = Std.transform(test)

poly3 = PolynomialFeatures(degree=3)
x_poly3 = poly3.fit_transform(X)
test_poly3 = poly3.fit_transform(test)

In [0]:
def submit_file(model):
  pred = model.predict(test_poly3)
  submit = pd.DataFrame({'ID':test_id,'Upvotes':pred})
  submit.to_csv('submit.csv',index=False)
  files.download('submit.csv')

## grid search cv
def gridsearchcv(xtrain,ytrain,model,params, scoring = None, fit_params = None):
  grid = GridSearchCV(model
                      ,params
                      ,scoring = scoring
                      ,fit_params = fit_params
                     # ,cv=StratifiedKFold(ytrain, n_folds=2,  shuffle=True)
                      ,cv = 4
                      ,verbose = 1
                      ,refit=True)
  grid.fit(xtrain,ytrain)
  
  best_parameters, score, _ = max(grid.grid_scores_, key=lambda x: x[1])
  print('Score:', np.sqrt(-score))
  for param_name in sorted(best_parameters.keys()):
      print("%s: %r" % (param_name, best_parameters[param_name]))
  return grid

def randomizedcv(xtrain,ytrain,model,params, scoring = None, fit_params = None):
  grid = RandomizedSearchCV(model, params, 
                   n_jobs=5, 
                   cv=3, 
                   scoring=scoring,
                   verbose=1,
                   refit=True)
  grid.fit(xtrain,ytrain)
  
  best_parameters, score, _ = max(grid.grid_scores_, key=lambda x: x[1])
  print('score:', np.sqrt(-score))
  for param_name in sorted(best_parameters.keys()):
      print("%s: %r" % (param_name, best_parameters[param_name]))
  return grid

In [25]:
cvs = cross_val_score(LinearRegression(),x_poly3,y,cv=3,scoring = 'neg_mean_squared_error')
cvs = [np.sqrt(-i) for i in cvs]
print(cvs)

#lr =  LinearRegression().fit(x_poly3,y)
#submit_file(lr_poly3)

[2122.3850820371536, 964.1365173429301, 1210.9512664271151]


In [10]:
# solve no space left on the device error
%env JOBLIB_TEMP_FOLDER=/tmp

env: JOBLIB_TEMP_FOLDER=/tmp


In [11]:
# xbg
params = {'min_child_weight':[4,5], 
          'gamma':[i/10.0 for i in range(3,6)],  
          'subsample':[i/10.0 for i in range(6,11)],
          'colsample_bytree':[i/10.0 for i in range(6,11)],
          'max_depth': [2,3,4]}

xgb = XGBRegressor()
model = randomizedcv(x_poly2,y,xgb,params,'neg_mean_squared_error')

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=5)]: Done  30 out of  30 | elapsed: 16.6min finished


score: 1232.8356240681817
colsample_bytree: 0.6
gamma: 0.3
max_depth: 4
min_child_weight: 4
subsample: 0.9


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:762: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)
